# Load Spectorgram Images into DataLoader for Training

In [ ]:
#train_dataloader = DataLoader()
#test_dataloader = DataLoader()
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

#TODO W
#data_path = './spectrograms' #looking in subfolder train
#data_path = './mfcc' #looking in subfolder train
data_path = './mel_spectorgrams'

yes_no_dataset = torchvision.datasets.ImageFolder(
    root=data_path,
    transform=transforms.Compose([transforms.Resize((201,81)),
                                  #transforms.Grayscale(num_output_channels=1),
                                  #transforms.RandomResizedCrop(200),
                                  #transforms.Pad(1, fill=10, padding_mode='constant'),
                                  transforms.ToTensor(),
                                  
                                  #transforms.Lambda(lambda y: torch.zeros(
                                  #10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                                  ])
)
print(yes_no_dataset)
print(yes_no_dataset[5][0].size())

In [ ]:
#split data to test and train
#use 80% to train
train_size = int(0.8 * len(yes_no_dataset))
test_size = len(yes_no_dataset) - train_size
yes_no_train_dataset, yes_no_test_dataset = torch.utils.data.random_split(yes_no_dataset, [train_size, test_size])

print(len(yes_no_train_dataset))
print(len(yes_no_test_dataset))

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    yes_no_train_dataset,
    batch_size=5,
    num_workers=2,
    shuffle=True
)

test_dataloader = torch.utils.data.DataLoader(
    yes_no_test_dataset,
    batch_size=5,
    num_workers=2,
    shuffle=True
)

In [ ]:
class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(51136, 50)
        self.fc2 = nn.Linear(50, 2)


    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #x = x.view(x.size(0), -1)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x,dim=1)
    
model = CNNet().to(device)

print(model)

In [ ]:
# cost function used to determine best parameters
cost = torch.nn.CrossEntropyLoss()

# used to create optimal parameters
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Create the training function

def train(dataloader, model, loss, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, Y) in enumerate(dataloader):
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = cost(pred, Y)
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')


# Create the validation/test function

def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (X, Y) in enumerate(dataloader):
            X, Y = X.to(device), Y.to(device)
            pred = model(X)

            test_loss += cost(pred, Y).item()
            #print(pred.argmax(axis=1))
            #print(Y.argmax(axis=1))
            #correct += (pred.argmax(1) == Y.argmax(1)).type(torch.float).sum().item()
            correct += (pred.argmax(1)==Y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size

    print(f'\nTest Error:\nacc: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f}\n')

In [ ]:
epochs = 5
#train_dataloader = DataLoader(spectorgram_train, batch_size=4, shuffle=True, num_workers=0)
#test_dataloader = DataLoader(spectorgram_train, batch_size =4, shuffle = True, num_workers=0)
for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------------')
    train(train_dataloader, model, cost, optimizer)
    test(test_dataloader, model)
print('Done!')